In [1]:
import os, pandas as pd, slr_pkg.clean_load_data as cld, slr_pkg.para as para
from slr_pkg.clean_load_data import open_table
from pathlib import Path

In [2]:
# Get current working directory
bp = Path(os.getcwd())

# Set base data directory.
dp  = bp / 'data'

# Set sample data directory.
edf_path = dp / 'geotracker_edf_results'
gama_path = dp / 'gama_results'

# Set location data directory.
geo_xy_path = dp / 'geotracker_xy'
gama_xy_path = dp / "gama_xy"
xy_path = dp / 'xy'

# Set results directory
results_path = bp / "results"

# Ask for county to gather data for.
# area = input('Enter county: ')
areas = ['Ventura', 'SanDiego', 'Kern', 'Imperial','SantaBarbara','LosAngeles']
# areas = ['LosAngeles']

In [35]:
### Depth to Water Data
# Load GAMA dtw data
# Create elev_path.
elev_path = bp / 'elevation'
print(elev_path, '\n')

# Dictionary of data types for gama_elev gama_elev for open_table().
gama_elev_dtypes = {
    'WELL NUMBER' : 'string',
    'DEPTH TO WATER' : 'float64',
    }

# Date column of gama_elev gama_elev for open_table().
gama_elev_date = ['MEASUREMENT DATE']

# Columns of gama_elev gama_elev for open_table().
gama_elev_cols = list(gama_elev_dtypes.keys()) + gama_elev_date


print('Loading GAMA groundwater elevations. \n')

# create list of files to open
gama_elev_files = elev_path.glob('**/*gama*.zip')
gama_elev_files = list(gama_elev_files)

# Use list comprehension to create a list of dataframes from the files list. Uses open_table() to open the files.
gama_elev_list = [open_table(i,dtypes = gama_elev_dtypes,date_cols = gama_elev_date, cols =gama_elev_cols) for i in gama_elev_files]
#print(gama_elev_list)

# Concatenate the list of dataframes into one dataframe if there are more than one.
if len(gama_elev_list) > 1:
    gama_elev = pd.concat(gama_elev_list)

else:
    gama_elev = gama_elev_list[0]

# Dict of attributes to rename.
gama_geo_dict = {
    'WELL NUMBER' : 'WID',
    'DEPTH TO WATER' : 'DTW',
    'MEASUREMENT DATE' : 'LOGDATE',
}
# Rename columns.
gama_elev = gama_elev.rename(columns=gama_geo_dict)

# Fix column formatting.
# gama_elev['LOGDATE'] = gama_elev['LOGDATE'].astype(str)
gama_elev['LOGDATE'] = pd.to_datetime(gama_elev['LOGDATE'].astype(str), errors='coerce', format='%Y-%m-%d')
gama_elev['WID'] = gama_elev['WID'].str.replace(' ', '')

# Create GID (group id) column. GID is the WID and LOGDATE concatenated.
gama_elev['GID'] = list(zip(gama_elev['WID'], gama_elev['LOGDATE']))
# Load Geotracker DTW data.
# Dictionary of data types for geo_elev geo_elev for open_table().
geo_elev_dtypes = {
    'GLOBAL_ID' : 'string',
    'FIELD_POINT_NAME' : 'string',
    'DTW' : 'float64',
    }

# Date column of geo_elev geo_elev for open_table().
geo_elev_date = ['GW_MEAS_DATE']

# Columns of geo_elev geo_elev for open_table().
geo_elev_cols = list(geo_elev_dtypes.keys()) + geo_elev_date

print('Loading Geotracker groundwater elevations. \n')

# create list of files to open
geo_elev_files = elev_path.glob('**/*Geo*.zip')
geo_elev_files = list(geo_elev_files)


# Use list comprehension to create a list of dataframes from the files list. Uses open_table() to open the files.
geo_elev_list = [open_table(i,geo_elev_dtypes,date_cols= geo_elev_date,cols =geo_elev_cols) for i in geo_elev_files]

# Concatenate the list of dataframes into one dataframe if there are more than one.
if len(geo_elev_list) > 1:
    geo_elev = pd.concat(geo_elev_list)

else:
    geo_elev = geo_elev_list[0]

# Create WID column.
geo_elev['WID'] = geo_elev['GLOBAL_ID'] + '-' + geo_elev['FIELD_POINT_NAME']

# Drop unnecessary columns.
geo_elev = geo_elev.drop(columns=['GLOBAL_ID', 'FIELD_POINT_NAME'])

# fix column formatting.
geo_elev['WID'] = geo_elev['WID'].str.replace(' ', '')

# Rename columns.
geo_elev = geo_elev.rename(columns={'GW_MEAS_DATE' : 'LOGDATE'})

# Fix column formatting.
geo_elev['LOGDATE'] = pd.to_datetime(geo_elev['LOGDATE'].astype(str), errors='coerce', format='%Y-%m-%d')
#geo_elev['LOGDATE'] = geo_elev['LOGDATE'].astype(str)

# Create GID (group id) column. GID is the WID and LOGDATE concatenated.
geo_elev['GID'] = list(zip(geo_elev['WID'], geo_elev['LOGDATE']))

e:\work\projects\coast_slr\scripts\slr_ground_water_quality_\elevation 

Loading GAMA groundwater elevations. 

Loading Geotracker groundwater elevations. 



In [36]:
# Concatenate gama_results and edf_results.
dtw = pd.concat([geo_elev, gama_elev])
dtw['dtw_units'] = 'ft'

# List of columns that require a value.
dtw_req_cols = ['WID','DTW','LOGDATE']

# Drops rows with missing values in required columns.
dtw = dtw.dropna(subset=dtw_req_cols)

# Drop duplicate GID rows.
dtw = dtw.drop_duplicates(subset=['GID'])

In [57]:
#locs = pd.read_csv(xy_path / 'all_xy_elev.csv')
c1 = ['LOGDATE', 'PARLABEL', 'PARVAL', 'PARVQ', 'UNITS', 'WID', 'GID',
       'SID', 'FIELD_PT_C', 'LATITUDE_x', 'LONGITUDE_x', 'chem_name',
       'comp_conc_val', 'exceedence', 'magnitude', 'DEM_ELV']

c2 = ['LOGDATE_x', 'PARLABEL', 'PARVAL', 'PARVQ', 'UNITS', 'WID_x', 'GID',
       'SID', 'FIELD_PT_C', 'LATITUDE_x', 'LONGITUDE_x', 'chem_name',
       'comp_conc_val', 'exceedence', 'magnitude', 'DEM_ELV', 'DTW', 'dtw_units']

c_dict = {
    'LOGDATE_x' : 'LOGDATE',
    'WID_x' : 'WID',
    'LATITUDE_x' : 'LATITUDE',
    'LONGITUDE_x' : 'LONGITUDE',
}


for area in areas:
    samples = pd.read_csv(dp / '{}_clean_samples_elev.csv'.format(area))
    print(samples.columns)
    samples = samples[c1]
    samples['LOGDATE'] = pd.to_datetime(samples['LOGDATE'].astype(str), errors='coerce', format='%Y-%m-%d')
    samples['GID'] = list(zip(samples['WID'], samples['LOGDATE']))
    samples['SID'] = list(zip(samples['GID'], samples['PARLABEL']))
    samples = samples.merge(dtw, left_on='GID', right_on='GID', how='left')
    print(samples.columns)
    samples = samples[c2]
    samples = samples.rename(columns=c_dict)
    samples.to_csv(dp / '{}_clean_samples_elev_dtw.csv'.format(area), index=False)

Index(['LOGDATE', 'PARLABEL', 'PARVAL', 'PARVQ', 'UNITS', 'WID', 'GID', 'SID',
       'FIELD_PT_CLASS', 'LATITUDE_x', 'LONGITUDE_x', 'chem_name',
       'comp_conc_val', 'exceedence', 'magnitude', 'OID_', 'FIELD_PT_C',
       'LATITUDE_y', 'LONGITUDE_y', 'DEM_ELV'],
      dtype='object')
Index(['LOGDATE_x', 'PARLABEL', 'PARVAL', 'PARVQ', 'UNITS', 'WID_x', 'GID',
       'SID', 'FIELD_PT_C', 'LATITUDE_x', 'LONGITUDE_x', 'chem_name',
       'comp_conc_val', 'exceedence', 'magnitude', 'DEM_ELV', 'LOGDATE_y',
       'DTW', 'WID_y', 'dtw_units'],
      dtype='object')
Index(['LOGDATE', 'PARLABEL', 'PARVAL', 'PARVQ', 'UNITS', 'WID', 'GID', 'SID',
       'FIELD_PT_CLASS', 'LATITUDE_x', 'LONGITUDE_x', 'chem_name',
       'comp_conc_val', 'exceedence', 'magnitude', 'OID_', 'FIELD_PT_C',
       'LATITUDE_y', 'LONGITUDE_y', 'DEM_ELV'],
      dtype='object')
Index(['LOGDATE_x', 'PARLABEL', 'PARVAL', 'PARVQ', 'UNITS', 'WID_x', 'GID',
       'SID', 'FIELD_PT_C', 'LATITUDE_x', 'LONGITUDE_x', 'chem_nam

In [7]:
cols = ['LOGDATE', 'PARLABEL', 'PARVAL', 'PARVQ', 'UNITS', 'LATITUDE', 'LONGITUDE', 'chem_name', 
 'DEM_ELV', 'DTW', 'dtw_units']

In [3]:
samples = pd.read_csv(dp / 'SanDiego_clean_samples_elev_dtw.csv')


In [10]:
samples = samples[(samples['FIELD_PT_C'] == 'MW') | (samples['FIELD_PT_C'] == 'MONITORING')].copy()

In [11]:
samples['FIELD_PT_C'].unique()

array(['MW', 'MONITORING'], dtype=object)

In [12]:
chems = para.conts11

for area in areas:

    samples = pd.read_csv(dp / '{}_clean_samples_elev_dtw.csv'.format(area))
    samples.dropna(subset=['DTW'], inplace=True)
    samples = samples[(samples['FIELD_PT_C'] == 'MW') | (samples['FIELD_PT_C'] == 'MONITORING')].copy()
    samples = samples[samples.loc[:,'PARLABEL'].isin(chems)]
    samples = samples[cols]

    samples.to_csv(results_path / '{}_all_dtw_11.csv'.format(area), index=False)

c:\Users\chief\.conda\envs\geoprj\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [73]:
for area in areas:

    # List of contaminants.
    chems = para.conts11

    samples = pd.read_csv(dp / '{}_clean_samples_elev_dtw.csv'.format(area))
    # subset of specific samples meeting parameters.
    spec_samples = samples.copy()
    spec_samples.dropna(subset=['DTW'], inplace=True)

    # Select spec_samples taken since 2012.
    spec_samples = spec_samples.loc[spec_samples['LOGDATE'] >= '2012-01-01']

    # Select samples with contaminants of interest.
    spec_samples = spec_samples[spec_samples['PARLABEL'].isin(chems)]

    # Create groups of spec_samples based on WID and PARLABEL(contaminant label).
    sample_groups = spec_samples.groupby(['WID'])['PARLABEL'].apply(list).reset_index()

    from collections import Counter


    def select_wells(row):
        wid = row['WID']
        counter = Counter(row['PARLABEL'])
        if len(counter) == len(chems):
            if all(i >= 4 for i in counter.values()):
                return  wid


    # Create mask of sample groups meeting parameter requirements.
    res = sample_groups.apply(select_wells, axis=1)

    # Use mask to select sample results from wells that meet parameter requirements.
    spec_samples = spec_samples[spec_samples['WID'].isin(res)]

    spec_samples.to_csv(results_path / '{}_spec_dtw_11.csv'.format(area), index=False)

c:\Users\chief\.conda\envs\geoprj\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
